

QUESTIONS:
What is our 30-day all cause readmission rate across hospitals our members are admitted to?
What percent of patients have a PCP visit within a week of discharge?
Do either vary by individual or group PCP's?

BONUS:
Which individual or PCP groups should we make an extra effort with to build a relationship? Which have the highest volume?
What percent of our patients do not have a PCP assigned or haven't seen their PCP recently?
Are there certain patients we should focus on (e.g. frequent fliers)?
From this, what could a version 1 of an actionable dashboard or intervention list look like to support the Transition Care Managers?

NOTES:

discharge date + 30 days: readmission true or false
count trues

In [1]:
import pandas as pd
import datetime as dt

In [2]:
members = pd.read_csv('../Data Files/NSS_GreatCare_Members.csv', parse_dates=[61,62])
admissions = pd.read_csv('../Data Files/NSS_Hospital_Inpatient_Admissions.csv', parse_dates=[8,9])
pcp_visits = pd.read_csv('../Data Files/NSS_PrimaryCare_Office_Visits.csv', parse_dates=[12])

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
admissions_dedup = admissions.sort_values(by=['Patient ID','Encounter Start Date', 'Encounter End Date']).drop_duplicates(subset=['Patient ID', 'Encounter Start Date'], keep = 'last')
admissions_dedup = admissions_dedup.sort_values(by=['Patient ID','Encounter End Date', 'Encounter Start Date']).drop_duplicates(subset=['Patient ID', 'Encounter End Date'], keep = 'first')
admissions.shape, admissions_dedup.shape

((6213, 23), (6149, 23))

In [4]:
def categorize_events(patient_admissions):
    patient_index_events = 0
    patient_readmission_events = 0
    last_index_event_discharge_date = None
    last_index_event_encounter_id = None
    readmission = False
    for ind, row in patient_admissions.iterrows():
        ##set first event as an intiial event or a non-qualifying readmission as initial event
        if patient_index_events == 0 or row['Encounter Start Date'] > last_index_event_discharge_date + dt.timedelta(days=30):
            patient_index_events += 1
            last_index_event_discharge_date = row['Encounter End Date']
            last_index_event_encounter_id = row['Encounter ID']
            readmission = False
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'index without readmission'
        ##check if qualifying readmission and change readmit status of most recently added initial event to true
        elif readmission == False:
            patient_readmission_events += 1
            readmission = True
            admissions.loc[admissions['Encounter ID'] == last_index_event_encounter_id, 'type'] = 'index w readmission'
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'initial readmission'
        ##check for duplicate readmission
        else:
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'extra readmission'
            
    return patient_index_events, patient_readmission_events
    

In [5]:
total_index_events = 0
total_index_events_leading_to_readmission = 0

for patient in admissions_dedup['Patient ID'].unique():
    patient_admissions = admissions_dedup[admissions['Patient ID']==patient]
    times_admitted = patient_admissions.shape[0]
    if times_admitted > 1:
        patient_admissions.sort_values(by = 'Encounter End Date', inplace=True, ignore_index=True)
        ind, re = categorize_events(patient_admissions)
        total_index_events += ind
        total_index_events_leading_to_readmission += re
    else:
        encounter_id = patient_admissions['Encounter ID'].values[0]
        admissions.loc[admissions['Encounter ID'] == encounter_id, 'type'] = 'index without readmission'
        total_index_events += 1
print("Total Index Events:",total_index_events) 
print("Index Events Leading to Readmission:", total_index_events_leading_to_readmission)
print(str.format("Readmission Rate: {}%",round(total_index_events_leading_to_readmission/total_index_events*100,2)))

<ipython-input-5-45bfff96a76a>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  patient_admissions = admissions_dedup[admissions['Patient ID']==patient]
<ipython-input-5-45bfff96a76a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_admissions.sort_values(by = 'Encounter End Date', inplace=True, ignore_index=True)


Total Index Events: 5536
Index Events Leading to Readmission: 527
Readmission Rate: 9.52%


In [6]:
admissions['type'].value_counts()

index without readmission    5009
index w readmission           527
initial readmission           527
extra readmission              86
Name: type, dtype: int64

In [7]:
admissions[['type']] = admissions[['type']].fillna(value='duplicate encounter')

In [8]:
admissions['type'].value_counts()

index without readmission    5009
index w readmission           527
initial readmission           527
extra readmission              86
duplicate encounter            64
Name: type, dtype: int64

In [9]:
total_hospital_visits = admissions_dedup.shape[0]
total_pcp_visits_in_7_days = 0

for ind, row in admissions_dedup.iterrows():
    patient = row['Patient ID']
    discharge_date = row['Encounter End Date']
    
    for i,r in pcp_visits[pcp_visits['Patient ID']==patient].iterrows():
        if r['Encounter Date'] <= discharge_date + dt.timedelta(days=7):
            total_pcp_visits_in_7_days += 1
            admissions.loc[admissions['Encounter ID']==row['Encounter ID'],'7 day followup']=True
            break
            
print('Total Hospital Visits: ', total_hospital_visits)
print('Total PCP Visits in 7 Days: ', total_pcp_visits_in_7_days)
print(str.format("PCP Visit Rate: {}%",round(total_pcp_visits_in_7_days/total_hospital_visits*100,2)))


Total Hospital Visits:  6149
Total PCP Visits in 7 Days:  5065
PCP Visit Rate: 82.37%


In [10]:
admissions[['7 day followup']] = admissions[['7 day followup']].fillna(value=False)

In [11]:
admissions.head()

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,type,7 day followup
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,...,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,index without readmission,True
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,...,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0,index without readmission,True
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,...,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,index without readmission,True
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,...,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,index without readmission,True
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,NaN,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0,index without readmission,True


In [12]:
pcp_visits.head(50)

,Encounter ID,Patient ID,Service Provider ID,Attending Provider ID,Provider Group Provider ID,Facility Provider ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Attending Provider NPI,Facility Provider NPI,Service Provider NPI,Provider Group NPI,Encounter Date,Is Qualifying Preventive Primary Care Service,Is Qualifying Primary Care Service Provider,Is Qualifying Primary Care Attending,Is Qualifying Primary Care Provider Group,Qualifying Preventive Primary Care HCPCS Code
0,COV|EE|DS1|10076|20110225|DS1|NPI|1487856779,DS1|10076,DS1|NPI|1487856779,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.487857e+09,1.043427e+09,2011-02-18,0,1,0,0,NaN
1,COV|EE|DS1|10088|20140222|DS1|NPI|1528030996,DS1|10088,DS1|NPI|1528030996,DS1|NPI|1528030996,DS1|NPI|1043427248,DS1|NPI|1902803315,DS1|1,DS1|5047W,1.528031e+09,1.902803e+09,1.528031e+09,1.043427e+09,2014-02-22,0,1,1,1,NaN
2,COV|EE|DS1|10165|20140907|DS1|NPI|1477579365,DS1|10165,DS1|NPI|1477579365,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.477579e+09,1.043427e+09,2014-08-31,0,1,0,0,NaN
3,COV|EE|DS1|10240|20130624|DS1|NPI|1821178088,DS1|10240,DS1|NPI|1821178088,DS1|NPI|1821178088,DS1|NPI|1407058787,DS1|NPI|1104842020,DS1|1,DS1|5047W,1.821178e+09,1.104842e+09,1.821178e+09,1.407059e+09,2013-06-17,0,1,1,1,NaN
4,COV|EE|DS1|10293|20160904|DS1|NPI|1609004191,DS1|10293,DS1|NPI|1609004191,NaN,DS1|NPI|1184722779,NaN,DS1|1,DS1|5047X,NaN,NaN,1.609004e+09,1.184723e+09,2016-09-04,0,1,0,0,NaN
5,COV|EE|DS1|1035|20130915|DS1|NPI|1164666715,DS1|1035,DS1|NPI|1164666715,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047Z,NaN,NaN,1.164667e+09,1.043427e+09,2013-09-08,0,1,0,0,NaN
6,COV|EE|DS1|1041|20100622|DS1|NPI|1720015902,DS1|1041,DS1|NPI|1720015902,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.720016e+09,1.043427e+09,2010-06-22,1,1,0,0,99396
7,COV|EE|DS1|10413|20151104|DS1|NPI|1194014357,DS1|10413,DS1|NPI|1194014357,NaN,DS1|NPI|1316025091,NaN,DS1|1,DS1|5047W,NaN,NaN,1.194014e+09,1.316025e+09,2015-11-04,1,1,0,0,99396
8,COV|EE|DS1|10440|20141124|DS1|NPI|1649221383,DS1|10440,DS1|NPI|1649221383,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.649221e+09,1.043427e+09,2014-11-17,0,1,0,0,NaN
9,COV|EE|DS1|10453|20130814|DS1|NPI|1669625612,DS1|10453,DS1|NPI|1770802746,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.770803e+09,1.043427e+09,2013-08-14,0,1,0,0,NaN


In [28]:
for ind, row in admissions.iterrows():
    patient = row['Patient ID']
    discharge= row['Encounter End Date']
    encounter = row['Encounter ID']
    mask = (pcp_visits['Patient ID']==patient) & (pcp_visits['Encounter Date']>=discharge)
    pcp_visits_subset=pcp_visits[mask].sort_values(by ='Encounter Date').reset_index()
    
    if len(pcp_visits_subset)==0:
        continue
    else:
        indiv_npi = pcp_visits_subset['Service Provider NPI'][0]
        group_npi = pcp_visits_subset['Provider Group NPI'][0]
        if indiv_NPI==group_NPI:
            admissions.loc[admissions['Encounter ID']==encounter, 'individual_npi'] = indiv_npi
        else:
            admissions.loc[admissions['Encounter ID']==encounter, 'individual_npi'] = indiv_npi
            admissions.loc[admissions['Encounter ID']==encounter, 'group_npi'] = group_npi
            

In [29]:
admissions.head(50)

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,type,7 day followup,individual_npi,group_npi
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,...,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,index without readmission,True,1.619980e+09,1.043427e+09
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,...,Compression of brain,NaN,NaN,0,46465.40,250.0,index without readmission,True,1.619200e+09,1.043427e+09
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,...,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,index without readmission,True,1.659394e+09,1.043427e+09
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,...,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,index without readmission,True,1.952538e+09,1.043427e+09
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0,index without readmission,True,1.316362e+09,1.043427e+09
5,HIA|DS1|10126|DS1|NPI|1902803315|1,DS1|10126,DS1|1,DS1|5047Q,DS1|NPI|1902803315,DS1|NPI|1346556891,1.346557e+09,1902803315,2010-04-07,2010-04-09,...,Methicillin resistant pneumonia due to Staphyl...,NaN,NaN,0,23344.52,250.0,index without readmission,True,1.568609e+09,1.043427e+09
6,HIA|DS1|10130|DS1|NPI|1427055839|1,DS1|10130,DS1|1,DS1|5047Z,DS1|NPI|1427055839,DS1|NPI|1316989247,1.316989e+09,1427055839,2014-06-01,2014-06-05,...,NaN,K35.2,Acute appendicitis with generalized peritonitis,0,21550.00,250.0,index without readmission,True,1.376562e+09,1.043427e+09
7,HIA|DS1|10132|DS1|NPI|1366545311|1,DS1|10132,DS1|1,DS1|5047X,DS1|NPI|1366545311,NaN,NaN,1366545311,2013-08-14,2013-08-16,...,Other specified cardiac dysrhythmias,NaN,NaN,0,10433.69,250.0,index without readmission,False,NaN,NaN
8,HIA|DS1|10132|DS1|NPI|1386741635|1,DS1|10132,DS1|1,DS1|5047X,DS1|NPI|1386741635,DS1|NPI|1417995317,1.417995e+09,1386741635,2013-03-30,2013-04-01,...,Poisoning by other specified psychotropic agents,NaN,NaN,0,9167.72,250.0,index without readmission,False,NaN,NaN
9,HIA|DS1|10135|DS1|NPI|1184722779|1,DS1|10135,DS1|1,DS1|5047W,DS1|NPI|1184722779,DS1|NPI|1396818134,1.396818e+09,1184722779,2013-08-20,2013-08-22,...,NaN,N85.02,Endometrial intraepithelial neoplasia [EIN],0,12066.00,250.0,index without readmission,True,NaN,NaN
